### Préparer la Base de Connaissances

In [2]:
faq_data = [
    {"question": "Comment suivre ma commande ?", "answer": "Vous pouvez suivre votre commande via votre espace client."},
    {"question": "Quels sont les modes de paiement disponibles ?", "answer": "Nous acceptons les paiements par carte, PayPal et Apple Pay."},
]

product_data = [
    {"id": 1, "name": "Laptop", "description": "Un ordinateur portable performant avec 16 Go de RAM."},
    {"id": 2, "name": "Smartphone", "description": "Un téléphone avec un écran AMOLED et une excellente caméra."},
]


### Indexer la Base de Connaissances

In [3]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd

# Charger un modèle d'encodage
encoder = SentenceTransformer('all-MiniLM-L6-v2')

# Indexer les données FAQ
faq_df = pd.DataFrame(faq_data)
faq_df['embedding'] = faq_df['question'].apply(lambda x: encoder.encode(x))

# Indexer le catalogue produits
product_df = pd.DataFrame(product_data)
product_df['embedding'] = product_df['description'].apply(lambda x: encoder.encode(x))


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Rechercher dans la Base

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

def find_most_similar(query, embeddings, data, top_k=1):
    query_embedding = encoder.encode(query).reshape(1, -1)
    similarities = cosine_similarity(query_embedding, np.vstack(embeddings))
    best_indices = similarities[0].argsort()[-top_k:][::-1]
    return [data.iloc[idx] for idx in best_indices]

# Exemple : Recherche dans la FAQ
query = "J'ai payé mon produit, c'est quoi la suite ?"
faq_result = find_most_similar(query, faq_df['embedding'], faq_df)
print(f'Question : {query}')
print(f"Réponse : {faq_result[0]['answer']}")

# Exemple : Recherche dans la desc des produits
query = "J'ai besoin d'un pc de développement logiciel ?"
prod_result = find_most_similar(query, product_df['embedding'], product_df)
print(f'Question : {query}')
print(f"Réponse : {prod_result[0]['description']}")


Question : J'ai payé mon produit, c'est quoi la suite ?
Réponse : Vous pouvez suivre votre commande via votre espace client.
Question : J'ai besoin d'un pc de développement logiciel ?
Réponse : Un ordinateur portable performant avec 16 Go de RAM.


### Filtrage Contextuel pour le Modèle Génératif

In [16]:
def filter_context(query, product_df, top_k=3):
    """
    Filtre les descriptions de produits pertinentes en fonction de la question.
    """
    product_results = find_most_similar(query, product_df['embedding'], product_df, top_k=top_k)
    return " ".join([res['description'] for res in product_results])


### Intégrer un Modèle Génératif

In [18]:
from transformers import pipeline

# Charger un modèle pré-entraîné
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

# Exemple : Génération de réponse
question = "Bonjour je cherche un ordinateur ?"
context = filter_context(query, product_df, top_k=3)
response = qa_pipeline(question=question, context=context)
print(f"Réponse générée : {response['answer']}")


Réponse générée : Un ordinateur portable performant avec 16 Go de RAM.


### Chatbot

In [19]:
def chatbot(query):
    # Étape 1 : Rechercher dans la FAQ
    faq_result = find_most_similar(query, faq_df['embedding'], faq_df, top_k=1)
    if faq_result and cosine_similarity(encoder.encode(query).reshape(1, -1), [faq_result[0]['embedding']])[0][0] > 0.75:
        return f"FAQ : {faq_result[0]['answer']}"
    
    # Étape 2 : Générer une réponse pour les requêtes complexes
    context = filter_context(query, product_df, top_k=3)
    response = qa_pipeline(question=query, context=context)
    if response['score'] > 0.7:
        return f"Generated: {response['answer']}"
    
    return "Je suis désolé, je ne peux pas répondre à cette question pour le moment."

# Tester le chatbot
print(chatbot("Quels produits ont 16 Go de RAM ?"))
print(chatbot("Comment suivre ma commande ?"))


Je suis désolé, je ne peux pas répondre à cette question pour le moment.
FAQ : Vous pouvez suivre votre commande via votre espace client.


In [22]:
def get_user_context(user_id):
    # Exemple : récupérer des commandes ou des interactions
    orders = [
        {"id": 1, "product": "Laptop", "status": "En cours de livraison"},
        {"id": 2, "product": "Smartphone", "status": "Livré"},
    ]
    return " ".join([f"Commande {order['id']} : {order['product']} ({order['status']})" for order in orders])

# Mise à jour du chatbot
def chatbot(query, user_id=None):
    # Étape 1 : Rechercher dans la FAQ
    faq_result = find_most_similar(query, faq_df['embedding'], faq_df, top_k=1)
    if faq_result and cosine_similarity(encoder.encode(query).reshape(1, -1), [faq_result[0]['embedding']])[0][0] > 0.75:
        return f"FAQ : {faq_result[0]['answer']}"
    
    # Étape 2 : Générer une réponse pour les requêtes complexes
    context = filter_context(query, product_df, top_k=3)
    if user_id:
        user_context = get_user_context(user_id)
        context = f"{context} {user_context}"
    
    response = qa_pipeline(question=query, context=context)
    if response['score'] > 0.7:
        return f"Generated: {response['answer']}"
    
    return "Je suis désolé, je ne peux pas répondre à cette question pour le moment."


In [23]:
# Test avec utilisateur
print(chatbot("Quels produits ont 16 Go de RAM ?", user_id=1))
print(chatbot("Comment suivre ma commande ?", user_id=1))


Je suis désolé, je ne peux pas répondre à cette question pour le moment.
FAQ : Vous pouvez suivre votre commande via votre espace client.
